In [ ]:
import pandas as pd
import numpy as np
import gensim
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset

In [ ]:
lyrics_df = pd.read_csv("./lyrics.tsv", delimiter='\t')

In [ ]:
print(lyrics_df.sample(3))

                                                lyrics  ...                                             morphs
405  샴푸가 되고 싶어 그대의 머리카락에 나 흘러내리며\n짙은 나의 향기로 그대를 감싸고...  ...  [['샴푸', '되', '싶', '그대', '머리카락', '나', '흘러내리', '...
116  사랑은 봄비처럼 다가와\n촉촉하게 날 흠뻑 적신다\n얼어붙어 있던 내 가슴을\n소리...  ...  [['사랑', '봄비', '다가와', '날', '적신다', '얼어붙', '있', '...
623  아직 남았을까\n못 잊을 이유가 남아 있을까\n나만 왜 이렇게\n바보같이 혼자 널 ...  ...  [['남', '잊', '이유', '남', '있', '나', '바보', '혼자', '...

[3 rows x 6 columns]


In [ ]:
lyrics_df.drop(['lyrics_sep', 'one_string'], axis=1, inplace=True)
print(lyrics_df)

                                                 lyrics  ...                                             morphs
0     나 아직 꿈을 꾸죠\n밤하늘 희미한 달빛처럼\n눈부셨던 기억 속에 그 사람\n어렴풋...  ...  [['나', '꿈', '꾸', '밤하늘', '달빛', '눈부셨', '기억', '속'...
1     약속해요 이 순간이 다 지나고\n다시 보게 되는 그날\n모든 걸 버리고 그대 곁에 ...  ...  [['약속', '순간', '지나', '보', '되', '그날', '걸', '버리',...
2     소슬한 바람이 마음에 불어\n나비인 듯 날아 마음이 닿는 곳\n맘 같지 않은 세상\...  ...  [['바람', '마음', '불', '나비', '인', '듯', '날', '마음', ...
3     그렇게 대단한 운명까진\n바란적 없다 생각했는데\n그대 하나 떠나간 내 하룬 이제\...  ...  [['운명', '바란', '적', '없', '생각', '그대', '하나', '떠나간...
4     이렇게 함께하는 편안한\n서로의 곁 연애할 때처럼 설레요\n밤이 내린 하늘엔 별이 ...  ...  [['하', '편안', '서로', '곁', '연애', '때', '설레', '밤', ...
...                                                 ...  ...                                                ...
1046  다가서면 멀어지는\n저  꿈결처럼\n그댄  어디로 가나요\n내 맘속에 살고 있는 그...  ...  [['다가서', '멀', '지', '꿈결', '그댄', '어디', '가', '내',...
1047  왠지 설레는 느낌\nMake me feel so high\n환한 미소가\n(I re...  ...  [['설레', '느낌', '환한', '미소'], ['빠져들', '싶', '너

In [ ]:
print(lyrics_df.columns)

Index(['lyrics', 'artist', 'age', 'morphs'], dtype='object')


In [ ]:
lyrics_df['morphs'][0]

In [ ]:
lyrics_tokens_df = pd.DataFrame(lyrics_df['morphs'])
print(lyrics_tokens_df)

                                                 morphs
0     [['나', '꿈', '꾸', '밤하늘', '달빛', '눈부셨', '기억', '속'...
1     [['약속', '순간', '지나', '보', '되', '그날', '걸', '버리',...
2     [['바람', '마음', '불', '나비', '인', '듯', '날', '마음', ...
3     [['운명', '바란', '적', '없', '생각', '그대', '하나', '떠나간...
4     [['하', '편안', '서로', '곁', '연애', '때', '설레', '밤', ...
...                                                 ...
1046  [['다가서', '멀', '지', '꿈결', '그댄', '어디', '가', '내',...
1047  [['설레', '느낌', '환한', '미소'], ['빠져들', '싶', '너', '...
1048  [['어디', '넌', '나', '지켜보', '지키', '못한', '약속', '잊'...
1049  [['어지러운', '날', '반', '가린', '돌아가', '시선', '가두', '...
1050  [['웃', '수', '있', '때', '웃', '둘', '걸', '너', '떠올리...

[1051 rows x 1 columns]


In [ ]:
def get_max_token_length(morphs):
  max_length = 0
  for tokens in morphs:
    if len(tokens) > max_length:
      max_length = len(tokens)

  return max_length

In [ ]:
lyrics_tokens_df['morphs'].to_list()

["[['나', '꿈', '꾸', '밤하늘', '달빛', '눈부셨', '기억', '속', '사람', '생각', '나', '네요'], ['바람', '흩날리', '꽃잎', '그대', '향기', '느끼', '스쳐', '지나간', '듯', '짧', '인연'], ['내', '전부', '인', '걸요', '같', '하늘', '아래', '그대', '있', '걸', '지워', '질까', '두려운', '거', '죠'], ['푸른', '바', '수평선', '걸', '수', '있', '나', '그대', '손', '놓', '않', '게', '하늘', '수놓', '진', '별빛'], ['그대', '숨결', '느끼', '운명', '그대', '남', '숨', '쉴', '수', '있'], ['같', '하늘', '아래', '그대', '있', '걸', '지워', '질까', '두려운', '거', '죠', '푸른', '바', '수평선'], ['걸', '수', '있', '나', '그대', '손', '놓', '않', '게', '같', '하늘', '아래', '그대', '있', '걸'], ['지워', '질까', '두려운', '거', '죠', '푸른', '바', '수평선', '걸', '수', '있', '나', '그대', '손', '놓', '않', '게']]",
 "[['약속', '순간', '지나', '보', '되', '그날', '걸', '버리', '그대', '곁', '서', '남', '길', '가리'], ['인연', '이', '하', '거부', '수', '없', '내', '생애', '이', '아름다운', '날', '올', '수', '있'], ['고달픈', '삶', '길', '당신', '선물', '인', '걸', '사랑', '녹슬', '않', '닦', '비출', '게요', '취한', '듯', '만남', '짧'], ['빗장', '열', '자리', '맺', '못한', '데', '후회', '않', '영원', '건', '없'], ['운명', '이', '하', '거부', '수', '없', '내', '생애', '이'

In [ ]:
lyrics_tokens_df['']

In [ ]:
sentences_train_len = [len(lyrics_tokens_df[i]) for i in range(len(lyrics_tokens_df))]

max_train_sentence_length = max(sentences_train_len)

In [ ]:
lyrics_tokens_df.map(lambda x: get_max_token_length(x)).max()

1

In [ ]:
best_model = torch.load("/content/drive/MyDrive/CUAI_Winter/classification_model.pth")